In [ ]:
import findspark
findspark.init('/opt/spark')
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("BDH-DATA-JOB") \
    .enableHiveSupport() \
    .getOrCreate()

print("Spark版本:", spark.version)

In [ ]:
! pip install geopandas shapely

In [2]:
id = 1
sql = """
SELECT lon, lat
FROM (
	SELECT lon, lat
	FROM time_series
	WHERE EXISTS (
		SELECT *
		FROM job
		WHERE job.id = {id}
			AND time_series.time BETWEEN job.start_time AND job.end_time
	)
) time_series;
""".format(id=id)
points_df = spark.sql(sql)
print(points_df.count())
points_df.show()


NameError: name 'spark' is not defined

In [1]:
# https://ncar.github.io/PySpark4Climate/tutorials/pyspark-geo-analysis/geopandas-and-spark/#Load-Data-in-a-GeoDataFrame
# 创建GeoDataFrame

lon_list = points_df.select("lon").rdd.flatMap(lambda x: x).collect()
lat_list = points_df.select("lat").rdd.flatMap(lambda x: x).collect()
import geopandas as gpd
from shapely.geometry import Point
from shapely.geometry import LineString
import matplotlib.pyplot as plt

# 创建几何对象
geometry = [Point(xy) for xy in zip(lon_list,lat_list)]

# 创建Geopandas的GeoDataFrame
gdf = gpd.GeoDataFrame(geometry=geometry)

fig,ax=plt.subplots()

# 将 GeoDataFrame 转换为 GeoSeries 对象
geo=gdf['geometry']

# 使用 GeoSeries 对象绘制线条
gpd.GeoSeries(geometry).plot(ax=ax)

# 显示图形
plt.show()

# # 创建曲线
# gdf['curve'] =gdf['geometry'].apply(lambda geom: LineString(geom.coords))

# # 删除重复的曲线
# gdf = gdf.drop_duplicates(subset=['group_column', 'curve'])

# # 将GeoDataFrame的几何列转换为WKT格式的字符串
# gdf['curve_wkt'] = gdf['curve'].apply(lambda x: x.wkt)

# print(gdf['curve_wkt'])


NameError: name 'points_df' is not defined

In [8]:
from shapely.geometry import LineString

print(f"LineString contains {len(gdf['geometry'].tolist())} points.")
linestring = LineString(gdf['geometry'].tolist())


LineString contains 999947 points.


In [9]:
sql = """
INSERT INTO gisdb.output (job_id, geoms)
VALUES (1, ST_GeomFromText('{linestring}', 4326));
""".format(linestring=linestring)
print(sql)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)

